<a href="https://colab.research.google.com/github/svvsaga/datascience_workshop/blob/peder/workshop_sesjon1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sesjon 1: "Extract and Load" med GCS og BigQuery

I denne sesjonen skal vi lære grunnleggende operasjoner for å få data inn i GCS og BigQuery. Vi ser også på hvordan vi kan hente data fra et API, og dytte disse inn i BigQuery.

# 1) Laste opp fil til GCS 
Det finnes flere måter å laster data inn til google cloud storage med ulike nivåer av kode-bruk:
- gjennom GCP cloud console (grafisk grensesnitt i nettleser)
- gjennom google cloud shell (terminal i nettleser)
- gjennom lokal installasjon av google cloud SDK, som brukes gjennom terminalen
- ved bruk av klientbliblioteker (f.eks. python client).

I stegene under kan du selv velge hvilken fremgangsmåte som passer best for deg. Vi forklarer både hvordan du utfører stegene i web-grensesnittet og via kommandolinja.

Dersom du ønsker å bruke kommandolinja, har vi under skrevet opp stegene for å klargjøre kommandolinja for bruk med GCP.



## Autentisering for bruk i python-kode

Vi skal senere kjøre python-kode som krever at du er autentisert mot GCP. Kjør kode-cellen under for å bli autentisert (trykk på play-knappen).

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated') 

Mens vi er i gang med oppsett kan vi like gjerne også definere prosjekt-IDen din i koden med en gang. Finn fram det fulle prosjektnavnet på prosjektet du har fått utdelt, og lim det inn under.

In [ ]:
project_id = "<prosjekt-navn i GCP>" # Endre denne
dataset_id = "workshop"              # Ikke endre denne

## Oppsett av kommandolinja (valgfri)

Dersom du ønsker å utføre stegene i denne notebooken via kommandolinja, utfør en av oppsettene under. Du trenger ikke å utføre begge.

### Oppsett Cloud shell
- Logg deg inn på GCP UI gjennom nettleseren din: https://console.cloud.google.com/ 
- Verifiser at prosjektet til workshop et valgt som arbeidsprosjekt
- Aktiver cloud shell ("Activate cloud shell" knapp i øverste høyre hjørnet i UI), vent til cloud shell er provisjonert

### Oppsett Google cloud SDK
- Last ned og installer Google cloud SDK for ditt system (i forkant av workshoppen): https://cloud.google.com/sdk/docs/install 
- initialiser cloud shell (verifiser at prosjektet er satt til ditt arbeids/workshop prosjekt)

## 1.1) Last ned eksempelfiler fra GCS
- Last ned 2 filer fra
 `gs://saga-trafikkdata-prod-pz8l_timetrafikkdata-ingest`. Dette er en bøtte som finnes i prosjektet `saga-trafikkdata-prod-pz8l` og som dere har fått rettigheter til å laste ned filer fra. Naviger til denne bøtta gjennom GCP web-grensesnittet. 
- Last ned de to filene:
 - `2021-03-23T09:31:31.945Z_2210582261344715.ndjson`
 - `2021-03-23T09:31:42.657Z_2210582990408255.ndjson`
- Dette kan også oppnås ved å bruke følgende cloud/SDK shell kommando:

```
gsutil cp gs://bucket-name path/to/file.csv
```

## 1.2) Lag en GCS bøtte
Lag en GCS bøtte der filene skal lastest opp. 
Dette kan gjøres enten gjennom grafisk user interface eller gjennom cloud shell/SDK

- I grafisk user interface naviger til Cloud Storage -> Browser og trykk på "Create bucket" 
- Velg et (globalt unik!) navn til bucket og konfigurer lagringsopsjoner
  - standard storage
  - multi-region, eu
  - uniform access

Det samme kan oppnås ved bruk av følgende cloud/SDK shell kommando:

```
gsutil mb -b -c standard -l EU gs://bucket-name
```

## 1.3) Laste opp filer til GCS bucket

Vi skal nå laste opp filene som vi lastet ned tidligere. Dette kan vi også gjøre via web-grensesnittet, eller via kommandolinja (f.eks. i cloud shell). Du velger selv fremgangsmåte.

Dersom du vil utføre dette via kommandolinja:

```
gsutil cp path/to/file.ndjson gs://bucket-name
```

Det er mulig å laste opp enkelte filter eller en liste med filter samtidig

```
gsutil cp path/to/*.ndjson gs://bucket-name
```

Til slutt verifiser at filene har blitt lastet opp, enten ved grafisk user interace (storage browser) eller gjennom cloud shell.

```
gsutil ls gs://bucket-name
```

En oversikt over gsutil kommandos finnes her: https://cloud.google.com/storage/docs/gsutil eller ved bruk av `gsutil help` i cloud shell terminal.

### Tips: Opplasting av store datamengder til GCS

På Saga-prosjektet har vi opplevd at det å bruke gsutil fungerer greit til middels store opplastinger til GCS, men når vi skulle laste opp flere terabyte med data til GCS ble det rett og slett for treigt. Da var det verktøyet [rclone som ble løsningen for oss](https://rclone.org/googlecloudstorage/). Vi anbefaler derfor rclone dersom du oppdager at gsutil bruker lang tid på opplastingen.

# 2) Importere data direkte fra GCS til BigQuery
Etter dataene har blitt lastet opp til GCS skal de ofte importeres til BigQuery til videre analyse. BigQuery er et datavarehus som kan lagre store datasett hvor man kan bruke SQL spørringer for å hente ut data.

BigQuery støtter en del formater direkte: 
- Avro
- CSV
- JSON
- ORC
- Parquet

Dataene kan importeres fra GCS bøtter, lokale filer (max 10MB upload per fil), google drive, BigTable eller genereres on-the-fly (lage en tom tabell).


## 2.1) Opprette et BigQuery-datasett

Her vil vi importere dataene vi har lastet opp til en GCS bøtte inn i BigQuery. BigQuery organiserer dataene i prosjekter, datasett og så tabeller. Før vi kan laste opp dataene i en tabell må vi derfor lage et nytt datasett.

**Det nye datasettet skal ha navnet workshop.**

Via web-grensesnittet kan dette gjøres ved å først navigere deg inn i BigQuery, velge ditt prosjekt og så klikke på "Create dataset". Location skal være den samme som for bøtta vi lagde, altså multi-region, EU.

Dersom du vil bruke kommandolinja, bruk følgende kommando.

```
bq --location=EU mk -d dataset_name
```






## 2.2) Importere data inn i ny tabell

Når datasett har blitt opprettet kan vi importere dataene fra GCS inn til en BigQuery tabell: `dataset_name.table_name`, hvor "dataset_name" altså skal være workshop.

I web-grensesnittet, velg datasettet "workshop" og trykk på "Create table".
Velg at filene skal lastes opp fra GCS. Gå via "Browse"-knappen for å finne bøtta og filene du vil laste opp. "Browse"-dialogen får det til å virke som at man må velge en fil, men det er i stedet mulig å skrive inn `*` i "name"-feltet for å matche alle filer, eller `*.ndjson` for å matche alle filer som slutter på `.ndjson`. Gjør dette slik at du får lastet opp begge filene.

Deretter kan du velge et tabellnavn (f.eks. "timestrafikkdata"). I tillegg må du huke av for "auto detect" for skjemaet. Deretter kan du velge "Create table".

Dersom du heller vil bruke kommandolinja:

```
bq --location=eu load --autodetect --source_format=NEWLINE_DELIMITED_JSON <DATASET>.<TABLE_ID> gs://<bucket_name>/2021-03-23T09:31:31.945Z_2210582261344715.ndjson
```

Det er også mulig å laste opp alle filer samtidig ved bruk av `*`:

```
bq --location=eu load --autodetect --source_format=NEWLINE_DELIMITED_JSON <DATASET>.<TABLE_ID> gs://<bucket_name>/*.ndjson
```

`--autodetect` er en flag som la BigQuery velge selv hvilken data type de ulike felter sannsynligvis har. Dette gjør ofte en rimelig bra jobb i første omgang og er veldig nyttig når man laster inn filer i JSON format som kan ha en kompleks, nestet struktur.

Alternativt er det mulig å angi en custom schema etter filnavn. Her kan det brukes en JSON-fil som definerer schema, eller en komma-separarert string av format `FELT_NAVN:DATA_TYPE,...`

## 2.3) Verifser import med en SQL spørring

Gjennom web-grensesnittet skal du nå kunne se en ny tabell. I alle fall dersom du oppdaterer siden. Verifiser at denne inneholder data ved å opprette og kjøre følgende SQL-spørring. Du må selv angi `<table_id>`.

```
SELECT *
FROM `workshop.<table_id>`
```

I denne seksjonen gjør vi en vri. Vi viser ikke hvordan du utfører en BigQuery-spørring via kommandolinja (men du kan gjøre det om du vil). I stedet viser vi hvordan du gjør det rett fra denne notebooken, både via å kjøre python-kode, og via ipython "inline magic".

Før vi kan hente ut data fra BigQuery via SQL spørringer så må vi definere prosjekt, dataset og tabel IDene.

In [ ]:
from google.cloud import bigquery

table_id = "<table id>"              # Endre denne til navnet på tabellen din

Her benytter vi oss av BigQuery "inline magic" kommandoer for å kjøre en SQL spørring direkte fra notebooken. Uheldigvis støtter ikke denne metoden å bytte ut "dataset_id" og "table_id" automatisk, så disse må du lime inn selv.Inline magic kommandoer kjøres ved bruk av `%%` øverst i notebook cellen. I tillegg angir vi et BigQuery prosjekt og en lokal python variable dataene skal lagres in.

In [ ]:
%%bigquery --project $project_id trip_df
SELECT *
FROM `workshop.<table_id>`

Her gjør vi det samme som over, men nå benytter vi oss av BigQuery klientbiblioteket for å kjøre den samme SQL spørringen:

In [ ]:
client = bigquery.Client(project=project_id)

trip_df = client.query(
    """
    SELECT *
    FROM `%s.%s`
    """ % (dataset_id, table_id)
).to_dataframe()

In [ ]:
trip_df.head()

## 2.4) Automatisk import fra GCS til BigQuery er ganske rå!

I skrivende stund har vi 370 GB++ med timesaggregert trafikkdata i bøtta `gs://saga-trafikkdata-prod-pz8l_timetrafikkdata-ingest` som vi så på tidligere. I denne oppgaven skal vi vise at det er null problem å laste opp hele dette datasettet til BigQuery, og det tar ikke engang særlig lang tid.

I web-grensesnittet, naviger deg tilbake til workshop-datasettet, og velg "Create table". Igjen skal datakilden være GCS. "Browse"-knappen lar deg ikke velge bøtter som ligger i andre prosjekter enn det du jobber i nå. Derfor må du denne gangen selv fylle inn `gs://saga-trafikkdata-prod-pz8l_timetrafikkdata-ingest/*.ndjson`. Ellers skal vi gjøre det samme som sist gang. Velg deg et tabellnavn og huk av for "auto detect" på skjema. Trykk så "Create table". Dette vil antakelig ta rundt 3-4 minutter. Du kan følge med på importjobbens status ved å navigere deg inn på "job history".

For å gjøre dette med kommandolinja kan vi gjøre akkurat det samme som sist gang, foruten at kildebøtta er ulik. Kommandoen blir altså noe slikt som:

```
bq --location=eu load --autodetect --source_format=NEWLINE_DELIMITED_JSON <DATASET>.<TABLE_ID> gs://<bucket_name>/*.ndjson
```


# 3) Importer data fra API

Av og til er det data fra et API man ønsker å utforske i BigQuery. Her viser vi hvordan man kan gjøre dette. Denne gangen skal vi ta en titt på [data fra trafikkdata-APIet](https://www.vegvesen.no/trafikkdata/api/?query=%7B%0A%20%20trafficRegistrationPoints%20%7B%0A%20%20%20%20id%0A%20%20%20%20name%0A%20%20%20%20location%20%7B%0A%20%20%20%20%20%20coordinates%20%7B%0A%20%20%20%20%20%20%20%20latLon%20%7B%0A%20%20%20%20%20%20%20%20%20%20lat%0A%20%20%20%20%20%20%20%20%20%20lon%0A%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%20%20%7D%0A%20%20%20%20%7D%0A%20%20%7D%0A%7D%0A). Trykk på "play"-knappen for å hente dataene i nettleseren. Her ser vi altså at dataene består av id, navn og posisjon for landets trafikkregistreringspunkter. Disse dataene skal vi bruke senere for å berike timestrafikkdataene med posisjon.

I denne seksjonen bruker vi python som kan kjøres rett i notebooken.

Først skal vi installere en "pre-release"-versjon av pakken gql, som lar oss gjøre spørringer mot API-et via GraphQL.

In [ ]:
!pip install --pre gql[all]

Så utfører vi det faktiske API-kallet, og printer det første trafikkregistreringspunktet i responsen.

In [ ]:
import json
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport
from gql.transport.requests import RequestsHTTPTransport

# Setup connection
transport = RequestsHTTPTransport(url="https://www.vegvesen.no/trafikkdata/api/", verify=True, retries=3)

# Create a GraphQL client using the defined connection
gql_client = Client(transport=transport, fetch_schema_from_transport=True)

# Provide a GraphQL query
query = gql(
    """
    {
      trafficRegistrationPoints {
        id
        name
        location {
          coordinates {
            latLon {
              lat
              lon
            }
          }
        }
      }
    }
    """
)

# Execute the query. The result is a native python dictionary.
data = gql_client.execute(query)
trafficRegistrationPoints = data["trafficRegistrationPoints"]

# Print the first traffic registration point in the list
print(json.dumps(trafficRegistrationPoints[0], indent=4))

Før vi laster opp dataene til BigQuery må vi lage den fulle identifikatoren til den nye tabellen vi skal opprette.

In [ ]:
table_id = "trafikkregistreringspunkter"

table_path = ".".join([project_id, dataset_id, table_id])
table_path

Dermed er vi klar til å laste opp dataene til BigQuery.

In [ ]:
bq_client = bigquery.Client(project=project_id)
load_job = bq_client.load_table_from_json(data["trafficRegistrationPoints"], table_path)
result = load_job.result()

Du kan nå verifisere at dataene har kommet inn i BigQuery. Hvordan du vil gjøre dette er opp til deg.

## 3.1) Frivillige oppgaver dersom du har tid til overs

**Oppgave 1:** Datastrukturen til `trafficRegistrationPoints` har unødvendig nøsting, i form av at lengde- og breddegrad ligger inn i "location.coordinates.latLon". Klarer du å flate ut strukturen i python-kode før dataene skrives til BigQuery? Vi ønsker altså å ende opp med at hvert element bare skal bestå av "id", "name", "lat" og "lon". **Hint:** bruk funksjonen `map(fun, iter)` til å mappe om hvert element i lista.

**Oppgave 2:** Når vi laster opp data til BigQuery med `load_table_from_json(...)` vil BigQuery forsøke å gjette skjemaet, og alle felter vil bli nullable (i motsetning til required). Det er mulig å angi skjemaet selv. Dette gjør at vi kan sette alle feltene til required, som vi ønsker at de skal være. Gjør dette. **Hint:** Send inn et ekstra argument `job_config=..` til `load_table_from_json(...)`, og sett `schema` i `LoadJobConfig`-objektet du sender inn. [Dokumentasjonen for denne funksjonen kan leses her.](https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.client.Client.html#google.cloud.bigquery.client.Client.load_table_from_json)

# Utdatert ekstramateriale

## Importere data med Dataflow inn i BigQuery

TODO: Tekst her

Først må vi installere Apache Beam i notebooken. Husk å trykke "Restart runtime" etter å ha installert python-modulene. Dette er nødvendig for at notebooken skal kunne bruke nyinstallerte moduler.

In [ ]:
!pip install 'apache-beam[interactive]' # For running beam interactively
!pip install 'apache-beam[gcp]'         # For running beam on GCP Dataflow


In [ ]:
# TODO: Gjere dette i starten
%env PROJECT_ID=saga-workshop-dtest-9hsr

Vi må også installere Apache Beam-pakker tilpasset bruk med GCP. Igjen, husk å trykke "Restart runtime" etterpå.

In [ ]:
# Ta vekk
!gcloud auth login

In [ ]:
# Ta vekk
!gcloud projects list

In [ ]:
!gcloud config set project $PROJECT_ID

In [ ]:
!gcloud auth application-default login

In [ ]:
import csv
import os

# Test

import apache_beam as beam
import apache_beam.runners.interactive.interactive_beam as ib
import apache_beam.io.fileio
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
from apache_beam.options import pipeline_options
from apache_beam.options.pipeline_options import GoogleCloudOptions


In [ ]:
options = pipeline_options.PipelineOptions()

project_id = os.environ['PROJECT_ID']

# Set the pipeline mode to stream the data from Pub/Sub.
options.view_as(pipeline_options.StandardOptions).streaming = False
options.view_as(GoogleCloudOptions).project = project_id

p = beam.Pipeline(InteractiveRunner(), options=options)

Dere skal nå få lov å sette sammen noen enkle Beam-pipelines selv. Under har vi inkludert en liste med 8 transformasjoner som utgjør deres "byggesett". Dere må selv velge ut de nødvendige transformasjonene basert på hva oppgaven spør etter.

- Finne filer som matcher et gitt mønster: `beam.io.fileio.MatchFiles(pattern))`
- Filtrere vekk elementer basert på en betingelse: `beam.Filter(condition_check_function)`
- Hente ut N tilfeldige elementer fra en større samling (PCollection): `beam.combiners.Sample.FixedSizeGlobally(N)`
- Gjøre om filstier til faktisk lesbare filer: `beam.io.fileio.ReadMatches()`
- Utføre en egendefinert transformasjon på hvert element. Passer best når output av transformasjonen er et enkelt element: `beam.Map(transform_function)`
- Utføre en egendefinert transformasjon på hvert element, hvor transformasjonen skal resultere i en liste med output-elementer. Denne vil da slå sammen alle output-elementer fra hver transformasjon i en felles liste: `beam.FlatMap(transform_function)`

En fullstendig liste med innebygde transformasjoner kan sees her: https://beam.apache.org/documentation/transforms/python/overview/


In [ ]:
file_pattern = "gs://saga-trafikkdata-prod-pz8l_timetrafikkdata-ingest/2021-04-10*" # April 10th, in the first 10 minutes of the 10th hour

file_paths = (p
  | "find files" >> beam.io.fileio.MatchFiles(file_pattern))

Vi har nå definert en minimal pipeline som finner filstier på alle filer i GCS som matcher mønsteret i `file_pattern`. For å sjekke at dette fungerer som det skal, kan vi bruke `ib.show`, som vi får fra den interaktive modulen til Apache Beam.

Men vi har ikke lyst å liste ut absolutt alle filene som matcher mønsteret. La oss heller plukke ut 10 tilfeldige filstier. Dette gjør vi med den innebygde "sample"-transformasjonen `beam.combiners.Sample.FixedSizeGlobally(n)`.

In [ ]:
file_samples = file_paths | "sample 10 files" >> beam.combiners.Sample.FixedSizeGlobally(10)
ib.show(file_samples)

Når vi har bekreftet at vi ser filstiene kan vi fortsette med pipelinen vår. I koden under har vi lagt til tre ekstra steg:

1. Et steg som klargjør filene til lesing
2. Et steg som transformerer innholdet i filene til en liste av python dictionaries
3. Et steg som skriver python dictionaries til BigQuery

Funksjonen `transform_file_to_objects` inneholder innmaten i steg 2. Slik den er nå vil den bare parse hver rad i filene til en python dictionary som gjenspeiler json-strukturen. Dette betyr at foreløpig vil pipelinen resultere i den samme datamodellen/skjema i BigQuery som vi fikk når vi lastet opp json-filene direkte.

Vi ønsker som sagt å også få med "ingest time", altså tidspunktet for vi mottok filene. Dette tidspunktet er en del av filnavnet. Dette ser vi dersom vi inspiserer filstiene vi fikk i steget over.


In [ ]:
import json

# This function transforms each line of an .ndjson file into a python dictionary.
# A python dictionary is a record type with keys and values, and is therefore a good fit for json data
# Later, we will send our dictionaries to Apache Beam's BigQuery library
def transform_file_to_objects(file):
  file_path = file.metadata.path
  file_contents = file.read_utf8()
  json_objects = []
  lines = file_contents.split('\n')
  for line in lines:
    if line and line.startswith("{"):
        # We use json.loads to parse each line (which contains json) into a python dictionary
        timestrafikkdata_row = json.loads(line)
        # timestamp_string = regex(...)
        #
        # TODO: Extract ingest time from the file_path parameter available in this function, and add it to timestrafikkdata_row
        # "Transform timestamp to date type and add it to timestrafikkdata_row"
        #
        # timestamp = Date(timestamp_string)
        # timestrafikkdata_row["ingest_time"] = timestamp
        json_objects.append(timestrafikkdata_row)
  return json_objects

# file.read_utf8()

write_results = (file_paths
  | "prepare files for reading" >> beam.io.fileio.ReadMatches()
  | "transform to objects" >> beam.FlatMap(transform_file_to_objects)
  | "write to BigQuery" >> beam.io.WriteToBigQuery("saga-workshop-dtest-9hsr:workshop.df5",
                                                   schema=beam.io.gcp.bigquery.SCHEMA_AUTODETECT,
                                                   write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND,
                                                   create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
                                                   custom_gcs_temp_location="gs://saga-workshop-dtest-9hsr_dataflow_files/temp",
                                                   method=beam.io.gcp.bigquery.WriteToBigQuery.Method.FILE_LOADS
                                                   ))

# p.run().wait_until_finish()

#  | "extract contents" >> beam.Map(lambda file: file.read_utf8()))

In [ ]:
from apache_beam.runners import DataflowRunner
from apache_beam.options.pipeline_options import WorkerOptions

df_options = pipeline_options.PipelineOptions()

# Set the project to the default project in your current Google Cloud
# environment.
df_options.view_as(GoogleCloudOptions).project = project_id
df_options.view_as(GoogleCloudOptions).region = 'europe-west4'
df_options.view_as(GoogleCloudOptions).service_account_email = 'dataflow@{}.iam.gserviceaccount.com'.format(project_id)
df_options.view_as(WorkerOptions).network = 'vpc-network'
#df_options.view_as(GoogleCloudOptions).subnetwork = 'regions/europe-west4/subnetworks/vpc-network'

# Storage for uploading Dataflow code and temporary files
dataflow_gcs_location = 'gs://saga-workshop-dtest-9hsr_dataflow_files'
df_options.view_as(GoogleCloudOptions).staging_location = '%s/staging' % dataflow_gcs_location
df_options.view_as(GoogleCloudOptions).temp_location = '%s/temp' % dataflow_gcs_location

runner = DataflowRunner()
runner.run_pipeline(p, options=df_options)

## Laste opp filer med klientbiblioteker.
- Filer kan også lastes opp ved bruk av client bibliotheker. En oversikt over støttede programmeringsspråk finnes her: https://cloud.google.com/storage/docs/reference/libraries 

- I cellen under finnes det et minimalt eksempel med python client bibliothek.

- For å kjøre python kode direkte fra denne notebook, må google cloud brukeren først autentiseres. 

- Vi bruker google.colab.auth python bibliothek til autentisering og google.cloud.storage til interaksjon med storage buckets

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated') 

In [ ]:
from google.cloud import storage
from google.cloud import bigquery
%load_ext google.colab.data_table

proj = # TODO define workshop-project
bucket =  # TODO define gs://bucket-name
file_name = # TODO define lokal file name
blob_name = # TODO define GCS blob name for the file
client = storage.Client(project = proj) # initialize client and set the billing project. NB: vi trenger ikke å gjenta credentials her.
bucket_object = client.get_bucket(bucket) # define the target bucket
blob = bucket.blob(blob_name) # make a file blob
blob.upload_from_filename(file_name) # upload the content from lokal file to GCS

NB: Det finnes klientbiblioteker til de fleste GCP verktøy/tjenester. F.eks i python: https://cloud.google.com/python/docs/reference 

# Outline
Her tar vi utgangspunkt i GPS-data fra Peder sin kjøretur fra Trondheim til Ørsta

1. Laste opp fil til GCS via terminalen (Cloud Shell/gsutil)
  - Nevne at det også kan gjøres via GUI og client libs
  - Nevne rclone for større datamengder
2. Importere data direkte fra GCS til BigQuery (Cloud Shell/bq)
  - Nevne at det også kan gjøres via GUI og client libs
3. Utføre en enkel BigQuery-spørring for å se at vi har fått inn data
4. Legge til datamapping i en Dataflow-jobb, og importere og mappe data inn i BigQuery
  - Kan bruke samme data som i pkt. 2, men at vi i DF-jobben forbedrer datamodellen, f.eks. ved å slå sammen to kolonner lat, long til en geography-kolonne
  - Bør vise Dataflow-GUI mens jobben kjører
5. Sjekke i BigQuery at dataene har komt inn på riktig format
6. Vise at dei kan laste inn alle dataene frå GCS til BigQuery via GUI
7. Seie litt om når Dataflow er riktig verktøy
8. Meir? Skal vi joine med geolokasjon i denne bolken eller seinare?
  - https://www.vegvesen.no/trafikkdata/api/?query=%7B%0A%20%20trafficRegistrationPoints%20%7B%0A%20%20%20%20id%0A%20%20%20%20name%0A%20%20%20%20location%20%7B%0A%20%20%20%20%20%20coordinates%20%7B%0A%20%20%20%20%20%20%20%20latLon%20%7B%0A%20%20%20%20%20%20%20%20%20%20lat%0A%20%20%20%20%20%20%20%20%20%20lon%0A%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%20%20%7D%0A%20%20%20%20%7D%0A%20%20%7D%0A%7D%0A

## Andre forslag:
- Bruke python libs + dataframes + pandas til noe (sesjon 1 eller 2?)
- Vurdere å vise Cloud Function som henter data og dytter det inn i Dataflow eller BigQuery?
- Andre verktøy: AirByte, FME og DBT

## Nytt forslag:
- Ta utgangspunkt i trafikkdata (saga-trafikkdata-prod-pz8l_timetrafikkdata-ingest)
- Gjere cirka det samme som før i steg 1, 2, 3
- Då ser vi at dataene for det første er veldig nøsta og at vi ikkje veit "ingest time" som er ein del av filnamna.
- Steg 4 (dataflow) vil då kunne gi verdi pga:
  1. Kan ha med ingesttime inn i BigQuery
  2. Kan hente ut berre totalt trafikkvolum, sidan vi ikkje bryr oss om resten